In [50]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,year,month,dayofmonth, to_timestamp, length, when,concat, lit, split,regexp_extract

**PROBLEM 4**: Supose Today is May 23, 2016. Display month-to-date count of returned orders <br>
NOTE: Month-to-date is a period starting at the beginning of the current month and ending at the current date. If today's date is May 23, 2016, then month-to-date shall be period between inclusive of May 01,2016 and May 23,2016

In [2]:
spark = SparkSession.builder.appName('learning').master('local').getOrCreate()
sc = spark.sparkContext

https://stackoverflow.com/questions/46594750/cast-column-containing-multiple-string-date-formats-to-datetime-in-spark

In [15]:
returnorder_df = spark.read.format('csv')\
                 .option('header','True')\
                 .option('delimiter',',')\
                 .option('inferSchema','True')\
                 .load('file:///home/saif/LFS/datasets/datasets_pavan/sales/order_returns.csv')

returnorder_df.printSchema()
returnorder_df.show(5)

root
 |-- ReturnDate: string (nullable = true)
 |-- TerritoryKey: integer (nullable = true)
 |-- ProductKey: integer (nullable = true)
 |-- ReturnQuantity: integer (nullable = true)

+----------+------------+----------+--------------+
|ReturnDate|TerritoryKey|ProductKey|ReturnQuantity|
+----------+------------+----------+--------------+
| 1/18/2015|           9|       312|             1|
| 1/18/2015|          10|       310|             1|
| 1/21/2015|           8|       346|             1|
| 1/22/2015|           4|       311|             1|
|  2/2/2015|           6|       312|             1|
+----------+------------+----------+--------------+
only showing top 5 rows



In [44]:
# by looking at the ReturnDate its not formatted properly we need to format them

In [43]:
formatted_date_returnorder_df = returnorder_df.withColumn('ReturnDate_foramatted', 
        when(col('ReturnDate').rlike("[\d]{1}/[\d]{1}/[\d]{4}")  == True, # example like this 1/1/2015
              concat(  lit("0"), 
                        split(col('ReturnDate'),'/').getItem(0),
                       lit("/"),
                       lit("0"), 
                        split(col('ReturnDate'),'/').getItem(1),
                       lit("/"), 
                       split(col('ReturnDate'),'/').getItem(2)
                    ) 
             )
        .when(col('ReturnDate').rlike("[\d]{2}/[\d]{1}/[\d]{4}")  == True, # example like this 10/1/2015
               concat(  split(col('ReturnDate'),'/').getItem(0),
                       lit("/"),
                       lit("0"), 
                        split(col('ReturnDate'),'/').getItem(1),
                       lit("/"), 
                       split(col('ReturnDate'),'/').getItem(2)
                    )
             )
        .when(col('ReturnDate').rlike("[\d]{1}/[\d]{2}/[\d]{4}") == True, # example like this 11/10/2015
              concat(lit("0"),col('ReturnDate'))
             )
        .otherwise(col('ReturnDate'))
                         )

formatted_date_returnorder_df.show(5)

+----------+------------+----------+--------------+---------------------+
|ReturnDate|TerritoryKey|ProductKey|ReturnQuantity|ReturnDate_foramatted|
+----------+------------+----------+--------------+---------------------+
| 1/18/2015|           9|       312|             1|           01/18/2015|
| 1/18/2015|          10|       310|             1|           01/18/2015|
| 1/21/2015|           8|       346|             1|           01/21/2015|
| 1/22/2015|           4|       311|             1|           01/22/2015|
|  2/2/2015|           6|       312|             1|           02/02/2015|
+----------+------------+----------+--------------+---------------------+
only showing top 5 rows



In [72]:
formatted_date_returnorder_df.select(
        year(to_timestamp(col("ReturnDate_foramatted"),'MM/dd/yyyy')).alias('Return_Year'),
        month(to_timestamp(col("ReturnDate_foramatted"),'MM/dd/yyyy')).alias('Return_month'),
        dayofmonth(to_timestamp(col("ReturnDate_foramatted"),'MM/dd/yyyy')).alias('Return_day'),
        col('ProductKey')
)\
.filter( (col('Return_Year') == 2015)  
         & (col('Return_month') == 5)  
         & ( (col('Return_day') >= 1) | (col('Return_day') < 23) ) 
      )\
.show(5)

+-----------+------------+----------+----------+
|Return_Year|Return_month|Return_day|ProductKey|
+-----------+------------+----------+----------+
|       2015|           5|         1|       312|
|       2015|           5|         6|       324|
|       2015|           5|         7|       351|
|       2015|           5|        13|       310|
|       2015|           5|        14|       313|
+-----------+------------+----------+----------+
only showing top 5 rows



In [63]:
df_1.printSchema()

root
 |-- Return_Year: integer (nullable = true)
 |-- Return_month: integer (nullable = true)
 |-- Return_day: integer (nullable = true)
 |-- ProductKey: integer (nullable = true)



In [66]:
df_1.show(5)

+-----------+------------+----------+----------+
|Return_Year|Return_month|Return_day|ProductKey|
+-----------+------------+----------+----------+
|       2015|           1|        18|       312|
|       2015|           1|        18|       310|
|       2015|           1|        21|       346|
|       2015|           1|        22|       311|
|       2015|           2|         2|       312|
+-----------+------------+----------+----------+
only showing top 5 rows



In [73]:
df_1.where(df_1.Return_Year == 2016).show(5)

Py4JJavaError: An error occurred while calling o887.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 51.0 failed 1 times, most recent failure: Lost task 0.0 in stage 51.0 (TID 51, 10.0.2.15, executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '010/03/2015' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.$anonfu

In [46]:
formatted_date_returnorder_df.show(10)

+----------+------------+----------+--------------+---------------------+
|ReturnDate|TerritoryKey|ProductKey|ReturnQuantity|ReturnDate_foramatted|
+----------+------------+----------+--------------+---------------------+
| 1/18/2015|           9|       312|             1|           01/18/2015|
| 1/18/2015|          10|       310|             1|           01/18/2015|
| 1/21/2015|           8|       346|             1|           01/21/2015|
| 1/22/2015|           4|       311|             1|           01/22/2015|
|  2/2/2015|           6|       312|             1|           02/02/2015|
| 2/15/2015|           1|       312|             1|           02/15/2015|
| 2/19/2015|           9|       311|             1|           02/19/2015|
| 2/24/2015|           8|       314|             1|           02/24/2015|
|  3/8/2015|           8|       350|             1|           03/08/2015|
| 3/13/2015|           9|       350|             1|           03/13/2015|
+----------+------------+----------+--

In [11]:
from pyspark.sql.functions import coalesce, to_date

def to_date_(col, formats=("MM/dd/yyyy", "dd-MM-yyyy")):
    # Spark 2.2 or later syntax, for < 2.2 use unix_timestamp and cast
    return coalesce(*[to_date(col, f) for f in formats])

In [ ]:
from pyspark.sql.functions import coalesce, to_date, udf

def to_date_udf(col):
    udf(lambda z: )

In [12]:
returnorder_df.withColumn("pdt", to_date_("ReturnDate")).show(5)

Py4JJavaError: An error occurred while calling o79.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 9, 10.0.2.15, executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '1/18/2015' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.$anonfun$pars

In [8]:
returnorder_df.select(to_timestamp(col('ReturnDate'),'MM/dd/yyyy').alias('Return_Date')).show(5)

Py4JJavaError: An error occurred while calling o57.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7, 10.0.2.15, executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2/2/2015' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.$anonfun$parse